In [1]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2610401 sha256=e8592ae4af1724ec4eceeb333eb9600d8c09a9c377ccf44a0bf8c4be8d0dcc69
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


In [2]:
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 35.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [1]:
# !pip install flask flask-ngrok
!pip install -q pyngrok

In [ ]:
# =============================================================================
# PHẦN -1: CÀI ĐẶT THƯ VIỆN CẦN THIẾT
# =============================================================================
# Bỏ comment dòng dưới đây và chạy nếu bạn chưa cài đặt các thư viện
# !pip install flask pandas numpy scikit-learn tensorflow surprise tqdm pyngrok

import pandas as pd
import numpy as np
import random
from collections import deque
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from surprise import Reader, Dataset, SVD
from tqdm import tqdm
import os
import sys
import socket
import threading
import time

# Thư viện cho Google Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Thư viện cho API
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
from pyngrok.exception import PyngrokError

# =============================================================================
# PHẦN 0: CẤU HÌNH VÀ THAM SỐ
# =============================================================================
# Đường dẫn file (Tự động điều chỉnh nếu không ở trong Colab)
if IN_COLAB:
    DRIVE_PATH = '/content/drive/MyDrive/Colab Notebooks/Project'
else:
    # THAY ĐỔI ĐƯỜNG DẪN NÀY CHO MÁY TÍNH CÁ NHÂN CỦA BẠN
    DRIVE_PATH = './data' # Ví dụ: './data' hoặc 'D:/Projects/MovieRec/data'
    if not os.path.exists(DRIVE_PATH):
        os.makedirs(DRIVE_PATH)


MOVIE_METADATA_PATH = os.path.join(DRIVE_PATH, 'The Movies Dataset/movies_metadata.csv')
RATINGS_PATH = os.path.join(DRIVE_PATH, 'The Movies Dataset/ratings_small.csv')
MODEL_WEIGHTS_PATH = os.path.join(DRIVE_PATH, 'dqn_movie_rec_weights.weights.h5')


# Tham số mô hình
NUM_CANDIDATES = 200      # Số lượng ứng viên từ mô hình hybrid
TOP_N_SVD = 150           # Số lượng phim lấy từ SVD
TOP_N_CONTENT = 150       # Số lượng phim lấy từ Content-Based

# Tham số DQN
STATE_HISTORY_LENGTH = 5  # Số phim gần nhất có rating cao để tạo state
REWARD_THRESHOLD = 3.5    # Ngưỡng rating để tính reward dương
MAX_EPISODE_STEPS = 50    # Số bước tối đa trong một episode (cho huấn luyện offline)
EPISODES = 100            # (Cho huấn luyện offline)
BATCH_SIZE = 32
GAMMA = 0.95
LEARNING_RATE = 0.001
MEMORY_SIZE = 2000
EPSILON_DECAY = 0.995

# =============================================================================
# PHẦN 1: TẢI VÀ TIỀN XỬ LÝ DỮ LIỆU
# =============================================================================
print("Đang tải và xử lý dữ liệu...")

try:
    movies_df = pd.read_csv(MOVIE_METADATA_PATH, low_memory=False)
    ratings_df = pd.read_csv(RATINGS_PATH)
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file dữ liệu. Vui lòng kiểm tra lại đường dẫn.")
    print(f"Đường dẫn metadata mong muốn: {MOVIE_METADATA_PATH}")
    print(f"Đường dẫn ratings mong muốn: {RATINGS_PATH}")
    exit()

# --- Tiền xử lý movies_df ---
movies_df['id'] = pd.to_numeric(movies_df['id'], errors='coerce')
movies_df.dropna(subset=['id'], inplace=True)
movies_df['id'] = movies_df['id'].astype('int')
movies_df['overview'] = movies_df['overview'].fillna('')

# --- Lọc phim có trong bộ ratings_small ---
movie_ids_in_ratings = ratings_df['movieId'].unique()
small_movies_df = movies_df[movies_df['id'].isin(movie_ids_in_ratings)].copy()
small_movies_df.drop_duplicates(subset=['id'], inplace=True)
small_movies_df.reset_index(drop=True, inplace=True)

# --- Sắp xếp ratings theo thời gian ---
ratings_df = ratings_df.sort_values('timestamp').reset_index(drop=True)

print(f"Số lượng phim sau khi lọc: {len(small_movies_df)}")
print(f"Số lượng đánh giá: {len(ratings_df)}")
print("Xử lý dữ liệu hoàn tất.")

# =============================================================================
# PHẦN 2: XÂY DỰNG CÁC MÔ HÌNH FILTERING TRUYỀN THỐNG
# =============================================================================

# --- 2.1: Content-Based Filtering (Dựa trên TF-IDF) ---
print("\nĐang xây dựng mô hình Content-Based (TF-IDF)...")
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(small_movies_df['overview'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
movie_id_to_idx = pd.Series(small_movies_df.index, index=small_movies_df['id'])
print("Xây dựng Content-Based hoàn tất.")

# --- 2.2: Collaborative Filtering (Dựa trên SVD) ---
print("\nĐang xây dựng mô hình Collaborative Filtering (SVD)...")
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
svd = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02)
svd.fit(trainset)

print("Đang tính toán trước các dự đoán SVD (có thể mất vài phút)...")
anti_testset = trainset.build_anti_testset()
svd_predictions = svd.test(anti_testset)
from collections import defaultdict
svd_recs_precomputed = defaultdict(list)
for uid, iid, _, est, _ in svd_predictions:
    svd_recs_precomputed[uid].append((iid, est))

for uid, user_ratings in svd_recs_precomputed.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
print("Xây dựng SVD và tính toán trước hoàn tất.")

# --- 2.3: Xây dựng bộ lọc Popularity-Based ---
print("\nĐang xây dựng bộ lọc Popularity-Based...")
movie_rating_counts = ratings_df['movieId'].value_counts()
popular_movies_df = small_movies_df[small_movies_df['id'].isin(movie_rating_counts.index)][['id', 'title']].copy()
popular_movies_df['rating_count'] = popular_movies_df['id'].map(movie_rating_counts)
popular_movies_df = popular_movies_df.sort_values('rating_count', ascending=False)
print("Xây dựng Popularity-Based hoàn tất.")


# --- 2.4: Hàm tạo danh sách ứng viên ---
def get_hybrid_candidates(user_id, user_history_df, num_candidates=NUM_CANDIDATES):
    watched_movies_in_history = user_history_df['movieId'].values
    svd_recs = [movie_id for movie_id, score in svd_recs_precomputed.get(user_id, [])
                if movie_id not in watched_movies_in_history]

    content_recs = []
    last_liked_movie = user_history_df[user_history_df['rating'] >= REWARD_THRESHOLD].tail(1)

    if not last_liked_movie.empty:
        last_movie_id = last_liked_movie['movieId'].iloc[0]
        if last_movie_id in movie_id_to_idx:
            idx = movie_id_to_idx[last_movie_id]
            sim_scores = list(enumerate(cosine_sim[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:TOP_N_CONTENT + 1]
            movie_indices = [i[0] for i in sim_scores]
            content_movie_ids = small_movies_df['id'].iloc[movie_indices].tolist()
            content_recs = [mid for mid in content_movie_ids if mid not in watched_movies_in_history]

    combined_recs = svd_recs[:TOP_N_SVD] + content_recs
    final_candidates = list(dict.fromkeys(combined_recs))
    return final_candidates[:num_candidates]

# =============================================================================
# PHẦN 3, 4, 5: MÔI TRƯỜNG RL, TÁC NHÂN DQN VÀ HUẤN LUYỆN OFFLINE
# =============================================================================

class MovieRecEnv:
    def __init__(self, ratings_data, movies_data, movie_id_map, tfidf_vectors):
        self.ratings_df = ratings_data
        self.movies_df = movies_data
        self.movie_id_to_idx = movie_id_map
        self.tfidf_matrix = tfidf_vectors
        user_counts = self.ratings_df['userId'].value_counts()
        self.valid_users = user_counts[user_counts >= 20].index.tolist()
        self.state_space_size = self.tfidf_matrix.shape[1]
        self.action_space_size = NUM_CANDIDATES

    def _get_user_history(self, user_id):
        return self.ratings_df[self.ratings_df['userId'] == user_id]

    def _create_state(self, user_history_df):
        high_rated_movies = user_history_df[user_history_df['rating'] >= REWARD_THRESHOLD].tail(STATE_HISTORY_LENGTH)
        if high_rated_movies.empty:
            return np.zeros(self.state_space_size)
        movie_indices = self.movie_id_to_idx.reindex(high_rated_movies['movieId']).dropna().astype(int)
        if movie_indices.empty:
            return np.zeros(self.state_space_size)
        state_vectors = self.tfidf_matrix[movie_indices]
        return np.array(state_vectors.mean(axis=0)).flatten()

    def reset(self):
        self.candidate_movies = []
        while len(self.candidate_movies) < self.action_space_size:
            self.current_user_id = random.choice(self.valid_users)
            full_user_history = self._get_user_history(self.current_user_id)
            split_point = int(len(full_user_history) * 0.8)
            self.rating_history = full_user_history.iloc[:split_point]
            self.future_interact = full_user_history.iloc[split_point:]
            if self.future_interact.empty or self.rating_history.empty:
                continue
            self.candidate_movies = get_hybrid_candidates(self.current_user_id, self.rating_history, self.action_space_size)
        state = self._create_state(self.rating_history)
        return state

    def step(self, action_idx):
        recommended_movie_id = self.candidate_movies[action_idx]
        actual_rating_info = self.future_interact[self.future_interact['movieId'] == recommended_movie_id]
        reward = -0.5
        done = False
        if not actual_rating_info.empty:
            actual_rating = actual_rating_info['rating'].iloc[0]
            reward = actual_rating - REWARD_THRESHOLD
            if reward > 0:
                successful_record = actual_rating_info.iloc[[0]]
                self.rating_history = pd.concat([self.rating_history, successful_record], ignore_index=True)
        next_state = self._create_state(self.rating_history)
        return next_state, reward, done

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=MEMORY_SIZE)
        self.gamma = GAMMA
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = EPSILON_DECAY
        self.learning_rate = LEARNING_RATE
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        model = Sequential([
            Input(shape=(self.state_size,)),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size, memory_buffer=None):
        # Sử dụng memory nội bộ nếu không có buffer ngoài được cung cấp
        if memory_buffer is None:
            memory_buffer = self.memory

        if len(memory_buffer) < batch_size:
            return

        minibatch = random.sample(memory_buffer, batch_size)
        states = np.array([transition[0] for transition in minibatch]).reshape(batch_size, self.state_size)
        next_states = np.array([transition[3] for transition in minibatch]).reshape(batch_size, self.state_size)

        q_current = self.model.predict(states, verbose=0)
        q_next = self.target_model.predict(next_states, verbose=0)

        for i, (state, action, reward, next_state, done) in enumerate(minibatch):
            target = reward if done else reward + self.gamma * np.amax(q_next[i])
            q_current[i][action] = target

        self.model.fit(states, q_current, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        if os.path.exists(name):
            self.model.load_weights(name)
            self.update_target_model()
            print(f"Đã tải trọng số từ {name}")
        else:
            print(f"Không tìm thấy file trọng số {name}. Bỏ qua việc tải.")

    def save(self, name):
        self.model.save_weights(name)
        print(f"Đã lưu trọng số vào {name}")

def train_dqn_model():
    print("\nBắt đầu quá trình huấn luyện OFFLINE...")
    env_offline = MovieRecEnv(ratings_df, small_movies_df, movie_id_to_idx, tfidf_matrix)
    agent_offline = DQNAgent(env_offline.state_space_size, env_offline.action_space_size)

    for e in range(EPISODES):
        state = env_offline.reset()
        state = np.reshape(state, [1, env_offline.state_space_size])
        total_reward = 0
        pbar = tqdm(range(MAX_EPISODE_STEPS), desc=f"Episode {e+1}/{EPISODES}")
        for time_step in pbar:
            action = agent_offline.act(state)
            next_state, reward, done = env_offline.step(action)
            total_reward += reward
            next_state = np.reshape(next_state, [1, env_offline.state_space_size])
            agent_offline.remember(state[0], action, reward, next_state[0], done)
            state = next_state
            pbar.set_postfix({"Score": f"{total_reward:.2f}", "Epsilon": f"{agent_offline.epsilon:.2f}"})
        agent_offline.replay(BATCH_SIZE)
        if e % 5 == 0:
            agent_offline.update_target_model()
    print("Huấn luyện OFFLINE hoàn tất.")
    agent_offline.save(MODEL_WEIGHTS_PATH)

# =============================================================================
# PHẦN 6: CÁC HÀM ĐỀ XUẤT (Tái cấu trúc và bổ sung)
# =============================================================================

def get_most_popular_movies(num_recs=10):
    top_movies = popular_movies_df.head(num_recs)
    return top_movies[['id', 'title']].to_dict('records')

def get_content_based_recommendations(movie_id, num_recs=10):
    if movie_id not in movie_id_to_idx:
        return None
    idx = movie_id_to_idx[movie_id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:num_recs+1]
    movie_indices = [i[0] for i in sim_scores]
    recommended_movies = small_movies_df.iloc[movie_indices][['id', 'title']]
    return recommended_movies.to_dict('records')


def get_recommendations_for_user(user_id, agent_instance, env_instance, num_recs=10):
    user_history = env_instance._get_user_history(user_id)
    if user_history.empty:
        print(f"Không có dữ liệu cho người dùng {user_id}, trả về phim phổ biến.")
        return get_most_popular_movies(num_recs)

    candidate_movies = get_hybrid_candidates(user_id, user_history, NUM_CANDIDATES)
    if not candidate_movies:
        print(f"Không thể tạo ứng viên cho người dùng {user_id}, trả về phim phổ biến.")
        return get_most_popular_movies(num_recs)

    # Giai đoạn cold-start hoặc warm-start
    interaction_count = len(user_history)
    INTERACTION_THRESHOLD = 5
    if interaction_count <= INTERACTION_THRESHOLD:
        print(f"User {user_id} trong giai đoạn COLD START. Sử dụng hybrid SVD + Content-Based.")
        final_recs_ids = candidate_movies[:num_recs]
    else:
        print(f"User {user_id} trong giai đoạn WARM START. Sử dụng DQN để xếp hạng lại.")
        state = env_instance._create_state(user_history)
        state = np.reshape(state, [1, env_instance.state_space_size])
        q_values = agent_instance.model.predict(state, verbose=0)[0]

        # Chỉ xếp hạng các ứng viên hợp lệ
        num_valid_candidates = len(candidate_movies)
        candidate_q_values = [(candidate_movies[i], q_values[i]) for i in range(num_valid_candidates)]

        sorted_recommendations = sorted(candidate_q_values, key=lambda x: x[1], reverse=True)
        recommended_movie_ids = [movie_id for movie_id, q_value in sorted_recommendations]
        final_recs_ids = recommended_movie_ids[:num_recs]

    rec_df = small_movies_df[small_movies_df['id'].isin(final_recs_ids)][['id', 'title']]
    # Sắp xếp lại dataframe theo đúng thứ tự đề xuất
    rec_df['sort_order'] = rec_df['id'].apply(lambda x: final_recs_ids.index(x))
    rec_df = rec_df.sort_values('sort_order').drop('sort_order', axis=1)
    return rec_df.to_dict('records')


# =============================================================================
# PHẦN 7: KHỞI TẠO API VÀ LOGIC HUẤN LUYỆN ONLINE
# =============================================================================
app = Flask(__name__)

# --- CÁC BIẾN TOÀN CỤC CHO API VÀ HUẤN LUYỆN ONLINE ---
ONLINE_MEMORY = deque(maxlen=MEMORY_SIZE * 10) # Tăng kích thước cho nhiều user
env = None
agent = None

def background_online_training(agent_instance, memory_buffer, batch_size):
    """
    Hàm này chạy trong một thread riêng, liên tục huấn luyện agent.
    """
    print("🚀 Bắt đầu tiến trình huấn luyện online trong nền...")
    update_counter = 0
    while True:
        try:
            if len(memory_buffer) > batch_size * 5:
                agent_instance.replay(batch_size, memory_buffer=memory_buffer)
                print(f"🤖 Đã chạy 1 batch huấn luyện online. Kích thước bộ nhớ: {len(memory_buffer)}. Epsilon: {agent_instance.epsilon:.4f}")
                update_counter += 1

                # Cập nhật target model sau mỗi 100 batch
                if update_counter % 100 == 0:
                    agent_instance.update_target_model()
                    print("🎯 Đã cập nhật Target Model.")

                # Lưu model định kỳ sau mỗi 500 batch
                if update_counter % 500 == 0:
                    agent_instance.save(MODEL_WEIGHTS_PATH)

            time.sleep(5) # Huấn luyện mỗi 5 giây
        except Exception as e:
            print(f"Lỗi trong background training thread: {e}")
            time.sleep(30) # Chờ lâu hơn nếu có lỗi

def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

# =============================================================================
# PHẦN 8: ĐỊNH NGHĨA CÁC API ENDPOINT
# =============================================================================

@app.route('/popular_movies', methods=['GET'])
def recommend_popular():
    num_recs = request.args.get('num_recs', default=10, type=int)
    return jsonify(get_most_popular_movies(num_recs))

@app.route('/similar_movies', methods=['GET'])
def recommend_similar():
    movie_id = request.args.get('movie_id', type=int)
    num_recs = request.args.get('num_recs', default=10, type=int)
    if movie_id is None:
        return jsonify({"error": "Vui lòng cung cấp movie_id"}), 400
    recommendations = get_content_based_recommendations(movie_id, num_recs)
    if recommendations is None:
        return jsonify({"message": f"Không tìm thấy phim với ID {movie_id}"}), 404
    return jsonify(recommendations)

@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = request.args.get('user_id', type=int)
    num_recs = request.args.get('num_recs', default=10, type=int)

    if user_id is None:
        return jsonify({"error": "Vui lòng cung cấp user_id"}), 400
    if agent is None or env is None:
        return jsonify({"error": "Mô hình chưa được khởi tạo"}), 500

    user_history = env._get_user_history(user_id)
    state = env._create_state(user_history)
    recommendations_dict = get_recommendations_for_user(user_id, agent, env, num_recs)

    response = {
        "recommendations": recommendations_dict,
        "context_state": state.tolist()
    }
    return jsonify(response)


@app.route('/feedback/rate', methods=['POST'])
def receive_rating_feedback():
    data = request.json
    try:
        user_id = int(data['user_id'])
        movie_id = int(data['movie_id'])
        rating = float(data['rating'])
        state_at_recommendation = np.array(data['context_state'])
    except (KeyError, TypeError, ValueError):
        return jsonify({"error": "Dữ liệu không hợp lệ. Cần user_id, movie_id, rating, context_state"}), 400

    if agent is None or env is None:
        return jsonify({"error": "Mô hình chưa được khởi tạo"}), 500

    reward = rating - REWARD_THRESHOLD

    # LƯU Ý QUAN TRỌNG:
    # Ánh xạ movie_id về action_index là một bài toán phức tạp trong thực tế.
    # Giải pháp đúng đắn là khi gọi /recommend, bạn cần lưu lại danh sách `candidate_movies`
    # cho user đó (ví dụ trong Redis) và lấy lại ở đây để tìm index.
    # Để đơn giản hóa, chúng ta giả định action_index = 0.
    # ĐÂY LÀ PHẦN CẦN CẢI TIẾN TRONG HỆ THỐNG THỰC TẾ.
    action_index = 0

    # Tạo next_state
    user_history = env._get_user_history(user_id)
    new_rating_record = pd.DataFrame([{'userId': user_id, 'movieId': movie_id, 'rating': rating, 'timestamp': time.time()}])
    updated_history = pd.concat([user_history, new_rating_record], ignore_index=True)
    next_state = env._create_state(updated_history)

    # Định dạng lại state để lưu vào bộ nhớ
    state = np.reshape(state_at_recommendation, [1, env.state_space_size])[0]
    next_state_reshaped = np.reshape(next_state, [1, env.state_space_size])[0]

    ONLINE_MEMORY.append((state, action_index, reward, next_state_reshaped, False))

    print(f"✅ Đã nhận feedback từ user {user_id} cho phim {movie_id}. Reward: {reward:.2f}.")

    return jsonify({"message": "Cảm ơn phản hồi của bạn!"}), 200

@app.route('/feedback/click', methods=['POST'])
def receive_click_feedback():
    data = request.json
    try:
        user_id = int(data['user_id'])
        movie_id = int(data['movie_id'])
    except (KeyError, TypeError, ValueError):
        return jsonify({"error": "Dữ liệu không hợp lệ. Cần user_id, movie_id"}), 400

    print(f"🖱️ Ghi nhận: User {user_id} đã chọn/click vào phim {movie_id}")
    return jsonify({"message": "Ghi nhận thành công."}), 200


# =============================================================================
# PHẦN 9: KHỞI CHẠY ỨNG DỤNG
# =============================================================================
if __name__ == '__main__':
    print("\n" + "="*50)
    print("      KHỞI CHẠY HỆ THỐNG ĐỀ XUẤT PHIM")
    print("="*50)

    # Nếu file weight chưa có, hỏi người dùng có muốn huấn luyện không
    if not os.path.exists(MODEL_WEIGHTS_PATH):
        print(f"\nFile trọng số '{MODEL_WEIGHTS_PATH}' không tồn tại.")
        choice = input("Bạn có muốn huấn luyện mô hình (offline) không? (y/n): ").lower()
        if choice == 'y':
            train_dqn_model()
        else:
            print("Bỏ qua huấn luyện. API có thể không hoạt động đúng.")

    # Khởi tạo agent và env toàn cục để sử dụng trong API
    try:
        print("\nĐang tải mô hình và môi trường dùng chung cho API...")
        env = MovieRecEnv(ratings_df, small_movies_df, movie_id_to_idx, tfidf_matrix)
        agent = DQNAgent(env.state_space_size, env.action_space_size)
        agent.load(MODEL_WEIGHTS_PATH)
        print("✅ Tải mô hình và môi trường thành công.")
    except Exception as e:
        print(f"❌ Lỗi nghiêm trọng khi khởi tạo agent/env toàn cục: {e}")
        agent = None
        env = None

    if agent:
        # Tạo và khởi chạy thread huấn luyện nền
        training_thread = threading.Thread(
            target=background_online_training,
            args=(agent, ONLINE_MEMORY, BATCH_SIZE),
            daemon=True
        )
        training_thread.start()

    # Chạy Flask app với ngrok
    port = find_free_port()
    # THAY TOKEN CỦA BẠN VÀO ĐÂY
    NGROK_AUTH_TOKEN = "31feNPxFxKOPs42wHw7YgyR8NfD_5tJA9tvGSizVzksWmF23Y"
    if NGROK_AUTH_TOKEN:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    public_url = ngrok.connect(port)
    print(f"\n🚀 Server đang chạy!")
    print(f" * Public URL (Ngrok): {public_url}")
    print(f" * Local URL: http://127.0.0.1:{port}")
    print("\nCác endpoints có sẵn:")
    print(f"  - Đề xuất cá nhân hóa: {public_url}/recommend?user_id=<ID>")
    print(f"  - Phim tương tự:       {public_url}/similar_movies?movie_id=<ID>")
    print(f"  - Phim phổ biến:        {public_url}/popular_movies")
    print(f"  - Gửi đánh giá (POST):  {public_url}/feedback/rate")
    print(f"  - Gửi click (POST):     {public_url}/feedback/click")

    try:
        app.run(host='0.0.0.0', port=port, use_reloader=False)
    except KeyboardInterrupt:
        print("\n Tắt ứng dụng và lưu trọng số cuối cùng...")
        if agent:
            agent.save(MODEL_WEIGHTS_PATH)
        ngrok.disconnect(public_url)
        print("Đã lưu. Tạm biệt!")

Mounted at /content/drive
Đang tải và xử lý dữ liệu...
Số lượng phim sau khi lọc: 2830
Số lượng đánh giá: 100004
Xử lý dữ liệu hoàn tất.

Đang xây dựng mô hình Content-Based (TF-IDF)...
Xây dựng Content-Based hoàn tất.

Đang xây dựng mô hình Collaborative Filtering (SVD)...
Đang tính toán trước các dự đoán SVD (có thể mất vài phút)...
Xây dựng SVD và tính toán trước hoàn tất.

Đang xây dựng bộ lọc Popularity-Based...
Xây dựng Popularity-Based hoàn tất.

      KHỞI CHẠY HỆ THỐNG ĐỀ XUẤT PHIM

Đang tải mô hình và môi trường dùng chung cho API...
Đã tải trọng số từ /content/drive/MyDrive/Colab Notebooks/Project/dqn_movie_rec_weights.weights.h5
✅ Tải mô hình và môi trường thành công.
🚀 Bắt đầu tiến trình huấn luyện online trong nền...


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



🚀 Server đang chạy!
 * Public URL (Ngrok): NgrokTunnel: "https://27e39be67a3f.ngrok-free.app" -> "http://localhost:42201"
 * Local URL: http://127.0.0.1:42201

Các endpoints có sẵn:
  - Đề xuất cá nhân hóa: NgrokTunnel: "https://27e39be67a3f.ngrok-free.app" -> "http://localhost:42201"/recommend?user_id=<ID>
  - Phim tương tự:       NgrokTunnel: "https://27e39be67a3f.ngrok-free.app" -> "http://localhost:42201"/similar_movies?movie_id=<ID>
  - Phim phổ biến:        NgrokTunnel: "https://27e39be67a3f.ngrok-free.app" -> "http://localhost:42201"/popular_movies
  - Gửi đánh giá (POST):  NgrokTunnel: "https://27e39be67a3f.ngrok-free.app" -> "http://localhost:42201"/feedback/rate
  - Gửi click (POST):     NgrokTunnel: "https://27e39be67a3f.ngrok-free.app" -> "http://localhost:42201"/feedback/click
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:42201
 * Running on http://172.28.0.12:42201
INFO:werkzeug:Press CTRL+C to quit
